In [3]:
import tensorflow as tf
import numpy as np
import os
import time

path_to_file=('recipes.txt')
text=open(path_to_file,'rb').read().decode(encoding='utf-8')
print(text[:250])


No-Bake Nut Cookies
1 c. firmly packed brown sugar
 1/2 c. evaporated milk
 1/2 tsp. vanilla
 1/2 c. broken nuts (pecans)
 2 Tbsp. butter or margarine
 3 1/2 c. bite size shredded rice biscuitsIn a heavy 2-quart saucepan, mix brown sugar, nuts, evapo


In [4]:
print(f'Length of text: {len(text)} characters')
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

Length of text: 2390691 characters
86 unique characters


In [5]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
print(chars)

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


2022-07-15 22:54:51.185379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
ids_from_chars=tf.keras.layers.StringLookup(vocabulary=list(vocab),mask_token=None)
ids=ids_from_chars(chars)
print(ids)

<tf.RaggedTensor [[60, 61, 62, 63, 64, 65, 66], [83, 84, 85]]>


In [7]:
chars_from_ids=tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars=chars_from_ids(ids)
print(chars)

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


In [8]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)
print(text_from_ids(ids))

tf.Tensor([b'abcdefg' b'xyz'], shape=(2,), dtype=string)


In [9]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8')) 
print(all_ids) 

tf.Tensor([45 74 15 ... 16  2  2], shape=(2390691,), dtype=int64)


In [10]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

N
o
-
B
a
k
e
 
N
u


In [11]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
for seq in sequences.take(1):
    print(chars_from_ids(seq))
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

tf.Tensor(
[b'N' b'o' b'-' b'B' b'a' b'k' b'e' b' ' b'N' b'u' b't' b' ' b'C' b'o'
 b'o' b'k' b'i' b'e' b's' b'\n' b'1' b' ' b'c' b'.' b' ' b'f' b'i' b'r'
 b'm' b'l' b'y' b' ' b'p' b'a' b'c' b'k' b'e' b'd' b' ' b'b' b'r' b'o'
 b'w' b'n' b' ' b's' b'u' b'g' b'a' b'r' b'\n' b' ' b'1' b'/' b'2' b' '
 b'c' b'.' b' ' b'e' b'v' b'a' b'p' b'o' b'r' b'a' b't' b'e' b'd' b' '
 b'm' b'i' b'l' b'k' b'\n' b' ' b'1' b'/' b'2' b' ' b't' b's' b'p' b'.'
 b' ' b'v' b'a' b'n' b'i' b'l' b'l' b'a' b'\n' b' ' b'1' b'/' b'2' b' '
 b'c' b'.' b' '], shape=(101,), dtype=string)
b'No-Bake Nut Cookies\n1 c. firmly packed brown sugar\n 1/2 c. evaporated milk\n 1/2 tsp. vanilla\n 1/2 c. '
b'broken nuts (pecans)\n 2 Tbsp. butter or margarine\n 3 1/2 c. bite size shredded rice biscuitsIn a heav'
b'y 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.\n Stir over medium'
b' heat until mixture bubbles all over top.\n Boil and stir 5 minutes more. Take off heat.\n Stir in vani'
b'lla and cer

In [14]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
print(split_input_target(list("i hate idtech"))) 


(['i', ' ', 'h', 'a', 't', 'e', ' ', 'i', 'd', 't', 'e', 'c'], [' ', 'h', 'a', 't', 'e', ' ', 'i', 'd', 't', 'e', 'c', 'h'])


In [15]:
dataset = sequences.map(split_input_target) 
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'No-Bake Nut Cookies\n1 c. firmly packed brown sugar\n 1/2 c. evaporated milk\n 1/2 tsp. vanilla\n 1/2 c.'
Target: b'o-Bake Nut Cookies\n1 c. firmly packed brown sugar\n 1/2 c. evaporated milk\n 1/2 tsp. vanilla\n 1/2 c. '


In [16]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = (dataset .shuffle(BUFFER_SIZE) .batch(BATCH_SIZE, drop_remainder=True) .prefetch(tf.data.experimental.AUTOTUNE))
print(dataset)

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


In [17]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        
        if states is None:
            states = self.gru.get_initial_state(x)

        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

model = MyModel(vocab_size=len(ids_from_chars.get_vocabulary()), embedding_dim=embedding_dim, rnn_units=rnn_units)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1): 
    example_batch_predictions = model(input_example_batch) 
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") 

(64, 100, 87) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  22272     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  89175     
                                                                 
Total params: 4,049,751
Trainable params: 4,049,751
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

print(sampled_indices)

print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

[12 27 46 36 49 38 26 24 50 19 48  4  5 42  1 55 27  9 59 24 40 72 13 26
 15 20 56  3 83 69 19 75 55 50 25 32 77 11 71 53 51 38 66 54  9  5 11 45
 36 21 35 71 84 20 33 15 41 55 66 17 43 44 80 46 76 40 13 19 69 84 28 29
 15 67  1 10 43 55 14 61 56  4 79 30 39 77 50 27 86 19 63 51 58 26 47 64
 36  8 13 23]
Input:
 b'ir in 2 cups of water and remaining ingredients.\n Bring to a boil.\n Reduce heat and cover. Simmer 20'
Next Char Predictions:
 b'*9OERG86S1Q!"K\tX9\'`6Im+8-2Y xj1pXS7Ar)lVTGgW\'")NE3Dly2B-JXg/LMuOqI+1jy:;-h\t(LX,bY!t=HrS9~1dT\\8PeE&+5'


In [21]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss: ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()

Prediction shape:  (64, 100, 87)  # (batch_size, sequence_length, vocab_size)
Mean loss:  tf.Tensor(4.46551, shape=(), dtype=float32)


86.96536

In [ ]:
model.compile(optimizer='adam', loss=loss)
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath=checkpoint_prefix, save_weights_only=True)
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

tf.saved_model.save(model, 'rnnModel1')

In [27]:
json_model=model.to_json()

with open('fashionmnist_model.json', 'w') as json_file:
    json_file.write(json_model)

model.save_weights('rnnModel.h5')

In [38]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(values=[-float('inf')]*len(skip_ids),indices=skip_ids,dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)
    
    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8') 
        input_ids = self.ids_from_chars(input_chars).to_tensor()
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        predicted_logits = predicted_logits
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
        predicted_chars = self.chars_from_ids(predicted_ids)
        return predicted_chars, states
        
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
tf.saved_model.save(one_step_model, 'one_step')

In [ ]:
states=None
next_char = tf.constant(['\n\n'])
result = [next_char]
for n in range(1000):
    next_char, states =  one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)
    
result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

In [41]:
model1=tf.saved_model.load('rnnModel')

In [47]:
states=None
next_char=tf.constant(['ROMEO:\n'])
result=[next_char]

for n in range(100):
    next_char,states=model1.generate_one_step(next_char, states=states)
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))


ROMEO:
1 can mushrooms
 1 (8 oz.) pkg. cream cheese
 1 c. Parmesan cheese
 bacon and carrotsMix squash and 
